In [1]:
import numpy as np
from nltk import tokenize
import ngram

In [2]:
TRAIN_DATA_PATH = 'data/raw_train.txt'
with open(TRAIN_DATA_PATH, 'r') as f:
    text = f.readlines()

# Tokenize the text
tokenized_text = [tokenize.word_tokenize(line) for line in text]

In [3]:
test_split = int(len(tokenized_text) * 0.2)
train_corpus = tokenized_text[test_split:]
test_corpus = tokenized_text[:test_split]
len(train_corpus)

23394

In [4]:
li_ngram = ngram.NGRAM_Model(4, scoring_method='LI_grid_search')
laplace_gram = ngram.NGRAM_Model(4)

In [5]:
li_ngram.train(train_corpus)

In [6]:
laplace_gram.train(train_corpus)

In [7]:
li_perp = np.mean([li_ngram.perplexity(x) for x in test_corpus])

In [8]:
laplace_perp = np.mean([laplace_gram.perplexity(x) for x in test_corpus])

In [9]:
print(f'Perplexity of LI model: {li_perp}')
print(f'Perplexity of Laplace model: {laplace_perp}')

Perplexity of LI model: 85.05477676314015
Perplexity of Laplace model: 271.0038173144343


In [10]:
def translate_sents(generate_sentences):
    SPECIAL_TOKENS = [ngram.SENTENCE_BEGIN, ngram.SENTENCE_END]
    updated_sentences = [list(filter(lambda x: x not in SPECIAL_TOKENS, s)) for s in generate_sentences]
    # join sentences
    updated_sentences = [' '.join(s) for s in updated_sentences]
    return updated_sentences

In [11]:
laplace_gen = laplace_gram.generate(10)
formatted_laplace = translate_sents(laplace_gen)

In [12]:
li_gen = li_ngram.generate(10)
formatted_li = translate_sents(li_gen)

In [13]:
GEN_LAPACE_PATH = 'data/laplace_gen.txt'
GEN_LI_PATH = 'data/li_gen.txt'
with open(GEN_LAPACE_PATH, 'w') as f:
    f.write('\n'.join(formatted_laplace))
with open(GEN_LI_PATH, 'w') as f:
    f.write('\n'.join(formatted_li))

['Less <UNK> to gainsay what they did',
 'beguile Nature of her custom , so <UNK> he is Angelo',
 'any in Italy , and as many of',
 'BALTHASAR :',
 'Lords , knights , and gentlemen , what I should say rather ;',
 'Isabel ,',
 'GREMIO :',
 "could have seen 't , the woe had been universal .",
 'crutches ere he was born desire yet their life to',
 'Glad that you thus continue your resolve']